# Import

In [1]:
import csv
import os
import pickle
import sys

import numpy as np
import pandas as pd
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from sklearn.mixture import GaussianMixture
from smart_open import open
from stop_words import get_stop_words
from tqdm import tqdm

## Add configuration file

In [2]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [3]:
from ALL import config 
from util import *

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [5]:
data_type="AgNews"
vectorize_type = "sentenceBERT"

# Read data

In [6]:
df = pd.read_csv(
    f"../../Preprocessing/data/{data_type}/master.csv", index_col=0
)

In [8]:
with open(f"../../Preprocessing/data/{data_type}/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [11]:
label = df["class"].to_numpy()

In [9]:
vector_dims = config["vectorize"]["doc2vec"]["dims"]
model_nums = config["clustering"]["gmm"]["max_model_num"]
vector_model_num = config["vectorize"]["doc2vec"]["max_model_num"]
covariance_types = config["clustering"]["gmm"]["covariance_types"]

In [12]:
class Corpus:
    def __init__(self, path=None, texts=None):
        self.path = path
        self.texts = texts
        self.dictionary = Dictionary(texts)

    def __iter__(self):
        if self.path is not None:
            for line in open(self.path):
                # assume there's one document per line, tokens separated by whitespace
                yield self.dictionary.doc2bow(line.lower().split())
        else:
            for line in self.texts:
                yield self.dictionary.doc2bow(line)

In [13]:
texts = df.words_nonstop.progress_apply(lambda x: x.split(' ')).tolist()
corpus = Corpus(texts=texts)
dictionary = Dictionary(texts)
#     登場頻度80%以上の単語を削除
dictionary.filter_extremes()

100%|██████████| 1000/1000 [00:00<00:00, 293759.91it/s]


In [14]:
pred_path = f"../../Clustering/data/{data_type}/{vectorize_type}/GMM/pred/"

for vector_dim in tqdm(vector_dims):
    coherences = []
    for model_num in range(model_nums):
        coherence = []
        for covariance_type in covariance_types:
            pred = np.load(
                f"{pred_path}{vector_dim}/{covariance_type}/{model_num}.npy"
            )

            cm = CoherenceModel(
                topics=pred.reshape(1, -1),
                corpus=corpus,
                dictionary=dictionary,
                texts=texts,
                coherence="c_v",
            )
            coherence.append(cm.get_coherence())
        coherences.append(coherence)
        coherence_path = f"../data/{data_type}/{vectorize_type}/GMM/coherence/{vector_dim}.csv"
        os.makedirs(os.path.dirname(coherence_path), exist_ok=True)
        pd.DataFrame(coherences, columns=covariance_types).to_csv(coherence_path)

100%|██████████| 9/9 [01:03<00:00,  7.07s/it]
